In [1]:
!pip install -Uq openai-agents


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os 
from dotenv import load_dotenv , find_dotenv # For Loading environment variables
from agents.run import RunConfig

# Load environment variables from .env file
load_dotenv(find_dotenv())

# set SECRET_KEY environment variable
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
model="gemini-2.0-flash"
base_url="https://generativelanguage.googleapis.com/v1beta/openai/"

In [3]:
from agents import AsyncOpenAI , OpenAIChatCompletionsModel , set_tracing_disabled

set_tracing_disabled(True)

provider = AsyncOpenAI(
    api_key=GEMINI_API_KEY,
    base_url=base_url,
)

model = OpenAIChatCompletionsModel(
    openai_client=provider,
    model=model
)

config = RunConfig(
    model=model,
    tracing_disabled=True
)


In [43]:
from agents import Agent, GuardrailFunctionOutput, RunContextWrapper, Runner, TResponseInputItem, input_guardrail
from pydantic import BaseModel

class HomeworkCheatDetectionOutput(BaseModel):
    attempting_cheat: bool
    explanation: str

homework_cheat_guardrail_agent = Agent(
    name="Homework Cheat Detector",
    instructions=(
        "Determine if the user's query resembles a typical homework assignment or exam question, indicating an attempt to cheat. General questions about concepts are acceptable. "
        " Cheating: 'Fill in the blank: The capital of France is ____.',"
        " 'Which of the following best describes photosynthesis? A) Cellular respiration B) Conversion of light energy C) Evaporation D) Fermentation.'"
        " Not-Cheating: 'What is the capital of France?', 'Explain photosynthesis.'"
    ),
    output_type=HomeworkCheatDetectionOutput,
    model=model
)

@input_guardrail
async def cheat_detection_guardrail(
        ctx: RunContextWrapper[None], agent: Agent, input: str | list[TResponseInputItem]
) -> GuardrailFunctionOutput :
    
    detection_result = await Runner.run(homework_cheat_guardrail_agent, input)

    return GuardrailFunctionOutput(
        tripwire_triggered=detection_result.final_output.attempting_cheat,
        output_info=detection_result.final_output
    )

study_helper_agent = Agent(
    name="Study Helper Agent",
    instructions="You assist users in studying by explaining concepts or providing guidance, without directly solving homework or test questions.",
    model=model,
    input_guardrails=[cheat_detection_guardrail]
)


In [ ]:
from agents import InputGuardrailTripwireTriggered

try:
    result = await Runner.run(study_helper_agent , "What is the capital of France?." , run_config=config)
    print("Guardrail didn't tripwire")
    print("response" , result.final_output)
except InputGuardrailTripwireTriggered as e:
    print("HOmework Guardrail tripped")
    print("Exceptions details" , str(e))

HOmework Guardrail tripped
Exceptions details Guardrail InputGuardrail triggered tripwire


## SalesMan Agent Guardrail

In [69]:
class SalesManDetectionOutput(BaseModel):
    is_competitor: bool
    is_price: bool

salesman_detection_agent = Agent(
    name="Salesman Detector",
    instructions=("Your Job is to answer user queries about products and offer them to buy But Don't answer questions about competitors and Price or give personal opinions."
                  "If User Ask Question about Competitors or Price or give personal opinions then stop the conversation."
                  "Don't Tell about price of new product."
                  "If user Ask Who is your Competitor then stop the conversation."
                  "If user Ask about Price then stop the conversation."),
    model=model,
    output_type=SalesManDetectionOutput
)

@input_guardrail
async def check_saleman_detection_guardrail(
        ctx : RunContextWrapper[None] , agent : Agent , input : str | list[TResponseInputItem]
    ) -> GuardrailFunctionOutput: # type: ignore

    check_detection = await Runner.run(salesman_detection_agent , input)

    return GuardrailFunctionOutput(
        tripwire_triggered=check_detection.final_output.is_competitor,
        output_info=check_detection.final_output
    )


Salesman_agent = Agent(
    name="Salesman Agent",
    instructions="You are a Salesman Agent. Your job is to answer user queries about products and offer them to buy But Don't answer questions about competitors and Price or give personal opinions.",
    model=model,
    input_guardrails=[check_saleman_detection_guardrail]
)

In [73]:
from agents import InputGuardrailTripwireTriggered

try:
    result = await Runner.run(Salesman_agent , "What is your Competitor?." , run_config=config)
    print("Guardrail didn't tripwire")
    print("response" , result.final_output)
except InputGuardrailTripwireTriggered as e:
    print("HOmework Guardrail tripped")
    print("Exceptions details" , str(e))

HOmework Guardrail tripped
Exceptions details Guardrail InputGuardrail triggered tripwire
